# HDL Coder Support Package for RFSoc 2x2 XUP Board Introduction
----

<!-- div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div -->

This notebook presents an introduction to the RF Data
Converters (RF DCs) on the RFSoC2x2 board using MATLAB Generated IP Cores.

## Aims
* Describe the `xrfdc` Python package that is required to control and
  configure the RF DCs from Jupyter
* Describe the modified RFSubsysem to access the modified RF Hierarchy
* Investigate the new radio hierarchy in the base overlay, allowing the
  user to develop very simple to very complex RF designs. 
* Present the data inspection and visualisation of the RF DCs using 
  Plotly.

## Table of Contents
* [Introduction](#introduction)
* [Hardware Setup](#hardware-setup)
* [File Access Setup](#FileAccess-setup)
* [The Radio Hierarchy](#radio-hierarchy)
* [Transmitter and Receiver](#the-transmitter-and-receiver)
* [RF Data Inspection](#rf-data-inspection)
* [Conclusion](#conclusion)

## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.3, June 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_3/pg269-rf-data-converter.pdf)
* [Xilinx, Inc, "Vivado Design Suite: The AXI Reference Guide", UG1037, v4.0, June 2017](https://www.xilinx.com/support/documentation/ip_documentation/axi_ref_guide/latest/ug1037-vivado-axi-reference-guide.pdf)
* [HDL Coder](https://www.mathworks.com/products/hdl-coder.html?s_tid=srchtitle_HDL%20Coder_1)
* [Getting Started with the HDL Workflow Advisor](https://www.mathworks.com/help/hdlcoder/ug/using-the-hdl-workflow-advisor-window.html?s_tid=srchtitle_hdl%2520workflow%2520advisor_2)

Credits: This introdcution is based on the 01_rf_dataconverter_introdcution.ipynb from Xilinx and has been modifided for this example.

## Revision History

* v1.0 | 02/13/2022 | First notebook revision.

----

## Introduction <a class="anchor" id="introduction"></a>

An overview of the Zynq RFSoC device with MATLAB Overlay is shown below; there are
four major components:

* Processing System (PS)
* Programmable Logic (PL)
* MATLAB Overload
* RF Data Converters (RF DC, including ADC and DAC)

The RF data converters are significant features of the Zynq RFSoC
device as they interface directly to the PL.
This brings many advantages, including the ability to
perform direct RF sampling and low-latency processing.

The RFSoC2x2 platform consists of a xczu28dr-ffvg1517 RFSoC device, 
which contains the following RF DCs:

* 8x RF Analogue-to-Digital Converters (RF ADCs)
    * Sample rate of 4 GSPS
    * 12-bit conversion
* 8x RF Digital-to-Analogue Converters (RF DACs)
    * Sample rate of 6.5 GSPS
    * 14-bit conversion

<br><img src='data/MATLAB_System.png' align='left' style='left' height='50%' width='50%'/>

Let's take a closer look at the RF DC block. Starting with
the RF ADC tiles, you will notice that there are 4 tiles on this
particular RFSoC device. Each tile contains 2 RF ADC blocks, 
which can be used to receive, or Digital Down Convert (DDC), 
an analogue signal. Each RF ADC block contains:

* A gearbox FIFO
* 2x, 4x, or 8x decimator
* A complex mixer
* A Quadrature Modulation Correction (QMC) unit

The image below presents the typical RF ADC processing 
pipeline:

1. The RF ADC samples the input waveform to convert it
   into a digital signal.
2. A threshold detector can be employed to detect and record
   input amplitude levels. 
3. The QMC is used to offset potential imbalance in the
   quality of the received complex signal.
4. The I and Q mixer can mix the input signal to baseband.
5. The I and Q decimators is capable of 2x, 4x, or 8x
   decimation before interfacing the PL via the gearbox FIFO.

<img src='data/rfsystem_rfadc_block_overview.png' align='left' style='left' height='75%' width='75%'/>

The RF DAC block has a similar pipeline to the RF ADC;
however, it is in reverse this time for Digital Up Conversion (DUC).

1. The data to be transmitted is interfaced to the RF DAC block from the PL.
2. The digital signal is interpolated by a factor of 2x, 4x, or 8x.
3. The digital signal is transferred to the complex mixer to be mixed
   to the desired carrier frequency. 
4. After the QMC and coarse delay block, the signal can optionally
   be convolved with an inverse (anti) sinc filter to improve the
   roll-off of the first Nyquist zone.
5. The digital signal is then converted to analogue through the
   RF DAC sampler.


<img src='data/rfsystem_rfdac_block_overview.png' align='left' style='left' height='75%' width='75%'/>

In this notebook, we will be demonstrating many of the features
mentioned above via a simple loopback example.

## Hardware Setup <a class="anchor" id="hardware-setup"></a>

Your RFSoC2x2 board is dual-channel. Notice that there are only
4 SMAs on your board, labeled DAC1, DAC2, ADC1, and ADC2.
Only two sets of RF ADC and RF DAC channels are accessible on
the RFSoC2x2 board.

You should create the loopback connection using SMA cables
as shown below:

* Channel 0: DAC2 to ADC2
* Channel 1: DAC1 to ADC1

See the image below for a demonstration.

<img src='data/rfsoc2x2_connections.jpg' align='left' style='left' width='40%' height='40%'/>

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

In this demo we are transmitting signals via cables.
This device can also transmit wireless signals. 
Keep in mind that unlicensed transmission of wireless signals
may be illegal in your location. 
Radio signals may also interfere with nearby devices,
such as pacemakers and emergency radio equipment. 
If you are unsure, please seek professional support.
</div>

----

## File Access Setup <a class="anchor" id="FileAccess-setup"></a>

Files can be transfered to the RFSoC using an SSH browser session or windows file share. Below is an example of how to configure linux to use SSH server connection. NOTE: default xilinx passwork may have to be set.

<img src='data/MATLAB_connect_to_server.png' align='left' style='left' width='40%' height='40%'/>

## The Radio Hierarchy <a class="anchor" id="radio-hierarchy"></a>

We need to add custom control logic in the PL to communicate
with the RF ADC or RF DAC. The MATLAB RFSoC2x2 base overlay contains
such logic that will allow you to use the RF DCs. 
There are two major hierarchies - the transmitter and the receiver.

* **Amplitude Controller** - can apply a value to the input of the RF DAC.
The value can be set by writing to an AXI-Lite register. 
The register value can be mixed with the Numerically
Controlled Oscillator (NCO) in the fine mixer to create a tone. 
The tone is looped back round into the RF ADC.

* **I/Q DMA & Stream Synchronizer** - can transmit real-time I/Q data to the RF DAC.
The I/Q DMA channel sends I/Q data to the user IP Core for further processing. 
I/Q Data is synchronized such that I/Q samples arrive at the user IP on the same
clock cycles.

* **AXI Stream Switch** - selects the I/Q data source.
The AXI Stream switch allows the user to control where I/Q Data samples are generated
from, either the Amplitude Generator or the User IP Core can be selected independatly 
for either channel.

* **Packet Generator** - interfaces to the output of RF ADC block. 
The data is used to generate an [AXI-Stream packet](https://www.xilinx.com/support/documentation/ip_documentation/axi_ref_guide/latest/ug1037-vivado-axi-reference-guide.pdf).
The AXI-Stream packet is sent to an AXI Direct Memory
Access (DMA) core and transferred to DDR memory.

These IP cores are connected to each channel of the RFSoC2x2 board.
They are conveniently placed inside a hierarchical block, `radio`, 
as shown below.

In addition to the I/Q data stream to/from the RF Tiles, the Numerically Controlled 
Oscialltors (NCO) and Threshhold Detection signals have been provided to the User IP.
These signals can be used to dirctly configure the NCO's Frequency and Phase. They can
be ignored and the NCO can still be controlled from the PYNQ system. NOTE: both methods
of controlling the NCO work congruently and the last write to the NCO's control registers
takes effect. 

**NOTE:** If using the NCO control from User IP is is best to provide a control
register within the User IP to set the center operating frequency and only allow the IP
to adjust the frequency within a range of that center frequency. Otherwise, the system
could transmit in undeiscred frequency ranges.

<img src='data/MATLAB_radio_hierarchy.png' align='left' style='left' height='60%' width='60%'/>

Let's now download the base overlay and initialize the drivers.

In [ ]:
# MATLABOverlay contains the Hierarchy control code and is a 
# derivate work of the original rfsysstem from Xilinx.
from MATLABOverlay import MATLABOverlay

# Import the Example user IP Core interface code
import MATLABExample_ip_0

base = MATLABOverlay('./MATLABOverlay/base.bit', ignore_version=True)
myIP = base.OFDM_RCVR_ip_0

The RFSoC2x2 has a sophisticated clocking network, which can generate
low-jitter clocks for the RF DC Phase-Locked Loops (PLLs). The base overlay
has a simple method to initialize these clocks. Run the cell below to set
the LMK and LMX clocks to 122.88MHz and 409.6MHz, respectively.

In [ ]:
base.init_rf_clks()

Now we will investigate the `radio` hierarchy.

In [ ]:
# Display the base documentation
base?

We can see the `transmitter` and `receiver` hierarchies. 
The RF DC object is also available, however, we don't need this. The reason
is that each transmitter and receiver is automatically paired with their
associated RF DC tile and block.

The `radio` hierarchy also initializes all of the active RF ADC and RF DAC
blocks to sample at 4096 GSPS (the maximum sample rate for the RF ADC).
By default the RF DAC and RF ADC mixer frequencies are 1024 MHz, which is
in the center of the first Nyquist Zone.

## Transmitter Data Source Selection
The transmitter hierarchy contains a source selection switch. The switch can select the Amplitude Generator that genrates a constant Complex waveform or the MATLAB IP Core. Selection settings are:
  ```
  0 - Samples from Amplitude Controller
  1 - Samples from MATLAB IP Core
  ```

In [ ]:
# Select IP Core for this demo
for i in range(0, len(base.radio.transmitter.channel)):
    base.radio.transmitter.channel[i].source.selection = 1
    base.radio.transmitter.channel[i].source.disabled = False
    base.radio.transmitter.channel[i].source.commit()

myIP.adc_capture_reset()

## Transmitter and Receiver <a class="anchor" id="the-transmitter-and-receiver"></a>

The transmitter hierarchy contains two channels. Each channel contains an
*Amplitude Controller* connected directly to an RF DAC. We can list the
available channels of the transmitter using the `get_channel_description()`
method.

In [ ]:
base.radio.transmitter.get_channel_description()

The report above states that there are two active channels in the system.

The receiver hierarchy also contains two channels. The *Packet Generator*
arranges the data into AXI-Stream packets and sends these packets to DDR
memory via AXI DMA. Let's run the cell below.

In [ ]:
base.radio.receiver.get_channel_description()

There are several methods to read and write to RF block registers.
These methods are common to the RF DAC and RF ADC.

```python
BlockStatus
MixerSettings
QMCSettings
CoarseDelaySettings
NyquistZone
```

For example, `base.radio.transmitter.channel[0].dac_block.QMCSettings`
will display the QMC settings for the transmit DAC block.

For many of these settings, you can change them and they will impact the operation.

In [ ]:
base.radio.transmitter.channel[0].dac_block.MixerSettings['Freq'] = 500
base.radio.transmitter.channel[0].dac_block.MixerSettings['MixerMode'] = 2
base.radio.transmitter.channel[0].dac_block.MixerSettings

Another example you can try:
```python
base.radio.receiver.channel[0].adc_block.CoarseDelaySettings
```

To further investigate the RF ADC and RF DAC, you can run the code cell below.

In [ ]:
help(base.radio.transmitter.channel[0].dac_block)

More to try:
```python
help(base.radio.transmitter.channel[0].dac_tile)
help(base.radio.receiver.channel[0].adc_block)
help(base.radio.receiver.channel[0].adc_tile)
```
Many of these attributes / methods depend on the configuration
of the PL and the hardware. You will need to implement your own hardware 
design if you want to do something else.

## RF Data Inspection <a class="anchor" id="rf-data-inspection"></a>

Now that you have a fundamental understanding of how to control the RF DCs, 
we can inspect RF data using the radio hierarchy block. To make it more
interesting, we generate stimulus using the RF DAC block for each channel. 
Let's configure the mixer frequencies for each RF-DAC block channel below.

The RF-ADC block mixer frequency can also be tuned. It can be tuned via the
MATLAB IP core or via the MixerSettings on the channel.

In [ ]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as ipw

In [ ]:
# Command IP Core to tune ADC Frequency
myIP = base.OFDM_RCVR_ip_0
myIP.adc0_freq(0)
myIP.adc0_update()

#base.radio.receiver.channel[0].adc_block.MixerSettings['Freq'] = 1024

# Read back the new MixerSetting to show IP Core upoated
base.radio.receiver.channel[0].adc_block.MixerSettings

Firstly, we will define the numbers of samples to be transferred. 
The range of samples that can be transferred is between 16 to 32768.
Running the code cell below, will transfer complex data.

In [ ]:
number_samples = 1024
base.radio.receiver.channel[0].setup_buffer(number_samples)
base.radio.receiver.channel[1].setup_buffer(number_samples)

In [ ]:
# Initiate the capture
myIP.adc_capture_reset()
cdata = []
for i in range(0, 1):
    base.radio.receiver.channel[i].initiate()

# Create a range of x values used to generate data to send to DAC
x = np.linspace(0, 1, 2**10)

# Create I/Q data using constant amplitude
re_samples = np.ones(number_samples)*.5
im_samples = np.ones(number_samples)*.5
   
# Create rectangular window
window = np.ones(number_samples)

# Apply simple window rectangular
wre_samples = np.multiply(window, re_samples)
wim_samples = np.multiply(window, im_samples)

# Create complex waveform to send to Transmitter
samples = wre_samples + 1j * wim_samples

base.radio.transmitter.channel[0].transfer(samples)
cdata.append(samples)

# Wait for passthru core to detect and capture samples
for i in range(0, 1):
    print("Waiting on {0}".format(i))
    base.radio.receiver.channel[i].wait()
    base.radio.receiver.channel[i].stop()
    cdata.append(base.radio.receiver.channel[i].fetch())


Now that we have complex channel data in Jupyter, we can import plotly, numpy,
and ipwidgets to enable data visualization.

The data can be plotted using `go.Scatter` and `go.FigureWidget` for each channel. Ipywidgets allows each figure to be displayed at the same time.

In [ ]:
sample_frequency = 2048e6
figs = []

for i in range(0, 1):
    plt_re_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.real(cdata[i]), name='Real'))
    plt_im_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.imag(cdata[i]), name='Imag'))
    figs.append(go.FigureWidget(data = [plt_re_temp, plt_im_temp],
            layout = {'title': ''.join(['Time Domain Plot of DAC Channel ', 
                                         str(i)]), 
                      'xaxis': {'title' : 'Seconds (s)',
                                'autorange' : True},
                      'yaxis' : {'title' : 'Amplitude (V)'}}))
    
for i in range(1, len(cdata)):
    plt_re_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.real(cdata[i]), name='Real'))
    plt_im_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.imag(cdata[i]), name='Imag'))
    figs.append(go.FigureWidget(data = [plt_re_temp, plt_im_temp],
            layout = {'title': ''.join(['Time Domain Plot of ADC Channel ', 
                                         str(i-1)]), 
                      'xaxis': {'title' : 'Seconds (s)',
                                'autorange' : True},
                      'yaxis' : {'title' : 'Amplitude (V)'}}))

ipw.VBox(figs)

In [ ]:
from scipy import signal
#from scipy import fft, fftshift
from scipy.fftpack import fft, fftshift

window = np.blackman(number_samples)
freq = np.linspace(-1024, 1024-1/number_samples, number_samples)
samps = np.linspace(0, number_samples/sample_frequency, number_samples)

figs = []

for i in range(0, len(cdata)): 
    wcdata = np.multiply(window, cdata[i])
    
    #wind_re_plt = go.Scatter(x=samps, y=np.real(wcdata), name="ADC{0} I".format(i))
    #wind_im_plt = go.Scatter(x=samps, y=np.imag(wcdata), name="ADC{0} Q".format(i))
    #figs.append(go.FigureWidget(data=[wind_re_plt, wind_im_plt]))
    
    mag = 20 * np.log10(np.abs((fft(wcdata, number_samples))))
    fft_plt = go.Scatter(x=freq, y=mag, name="ADC{0}".format(i))
    
    figs.append(go.FigureWidget(data=[fft_plt]))
    
ipw.VBox(figs)

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has presented a simple introduction to the MATLAP IP Core Generation to access the RF Data Converters. 
The RF DCs were briefly described and a radio hierarchy in PL was explored.
RF data visualization was performed using `plotly`, `ipywidgets`, and `numpy` in Jupyter.


Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

Copyright (C) 2022 Brian G. Shea

SPDX-License-Identifier: BSD-3-Clause

----

----